## Face, People & Car Detection

* Understanding HAAR Cascade Classifiers
* Face & Eye detection
* Mini project _ car detection & pedestrian (body) detection

### HAAR Cascade Classifiers

앞의 사례에서 extracting features가 가능하고, 해당 features를 활용해 classify object가 가능하다는 걸 보였다.

HAAR Cascade Classifier도 위 원칙을 그대로 따르는데, different way of storing the features. 

이 features의 이름을 Haar Feature라고 한다. 얘네를 series of classifiers(cascade)에 넣어서 identify objects in an image.

Detecting one particular object에 매우 효율적이며, parallel하게 여러 개를 학습시켜서 다른 것들을 detecting하는 것도 가능하다. 

ex) detecting eyes and face together.

#### Haar classifier explained.

1. HAAR classifiers are trained using lots of positive images(with the objects present) and negative images(without the object present)
    - 예컨대 얼굴 인식이라면 얼굴이 있는 사진은 Positive, 얼굴 없는 사진은 negative.

2. 이제 이미지에서 HAAR Features를 뽑아내야 한다. 
    - HAAR Feature는 간단히 말하면, sum of pixel intensities under rectangular windows. 여기서 windows로 다양한 형태의 rectangular를 사용한다. edge feature, line feature 등등... 
    - 다만, 이렇게 다양한 window로 feature를 뽑아내면 너무 많은 양의 feature가 만들어진다. 그래서 Integral images라는 방식을 사용하는데, 얘는 computed with 4 array references.
    - feature 수를 줄여도, 대다수의 features는 별로 쓸모가 없다. 얼굴 인식이라면, 얼굴을 정확히 가리키는 feature는 극히 일부일 거니까. 그래서 Boosting 기법이 들어왔고, 여기서는 AdaBoost를 사용한다. 간단히 말해 incorrect classification에는 penalty를 부과해서 classifier의 성능을 높이는 것. 이렇게 무의미한 feature를 상당부분 줄여냈다.
    - 줄여냈다고 해도, 여전히 no real value인 feature가 많다. 정확히는 information의 양 차이가 나는 feature는 존재할 것. 그래서 사용한 방법으로 "check whether the region can potentially have a face". 먼저 이것부터 확인하면, 모든 feature를 일일이 연산해서 대조할 필요는 없어진다.
    - 이런 방식이 cascade of classifier이며, Viola Jones method의 경우 38 stage에 걸쳐 진행된다고 함.


### Cascade classifier를 사용해 face / eye detection.

pretrained된 OpenCV Classifier를 사용할 예정. 깃허브에도 있고, 강의자료에도 제공돼 있다.

haarcascade_eye.xml 과 haarcascade_frontalface_default.xml 사용.

#### cascade classifier flow 사용법

1. load classifier
2. pass img to classifier / detector
3. get location / ROI(region of interest) for detected objects
4. draw rectangle over detected objects.



In [1]:
## Face Detection

import numpy as np
import cv2

# We point OpenCV's CascadeClassifier function to where our 
# classifier (XML file format) is stored
# classifier 생성.
face_classifier = cv2.CascadeClassifier('./MasteringComputerVision-V1.03/Master OpenCV/Haarcascades/haarcascade_frontalface_default.xml')

# Load our image then convert it to grayscale
image = cv2.imread('./MasteringComputerVision-V1.03/Master OpenCV/images/Trump.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Our classifier returns the ROI of the detected face as a tuple
# It stores the top left coordinate and the bottom right coordiantes

# 리턴값은 array. multiple location of face 반환 (얼굴이 여러 개일 경우)
faces = face_classifier.detectMultiScale(gray, 1.3, 5)

# When no faces detected, face_classifier returns and empty tuple
if faces is ():
    print("No faces found")

# We iterate through our faces array and draw a rectangle
# over each face in faces
for (x,y,w,h) in faces:
    cv2.rectangle(image, (x,y), (x+w,y+h), (127,0,255), 2)
    cv2.imshow('Face Detection', image)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()


In [4]:
## Face + Eye Detector

import numpy as np
import cv2
 
face_classifier = cv2.CascadeClassifier('./MasteringComputerVision-V1.03/Master OpenCV/Haarcascades/haarcascade_frontalface_default.xml')
eye_classifier = cv2.CascadeClassifier('./MasteringComputerVision-V1.03/Master OpenCV/Haarcascades/haarcascade_eye.xml')
 
img = cv2.imread('./MasteringComputerVision-V1.03/Master OpenCV/images/Trump.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_classifier.detectMultiScale(gray, 1.3, 5)

# When no faces detected, face_classifier returns and empty tuple
if faces is ():
    print("No Face Found")

for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(127,0,255),2)
    cv2.imshow('img',img)
    cv2.waitKey(0)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_classifier.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(255,255,0),2)
        cv2.imshow('img',img)
        cv2.waitKey(0)
    
cv2.destroyAllWindows()



#### Live face / eye detection

In [14]:
import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier('./MasteringComputerVision-V1.03/Master OpenCV/Haarcascades/haarcascade_frontalface_default.xml')
eye_classifier = cv2.CascadeClassifier('./MasteringComputerVision-V1.03/Master OpenCV/Haarcascades/haarcascade_eye.xml')

def face_detector(img, size=0.5):
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img
    
    for (x,y,w,h) in faces:
        x = x - 50
        w = w + 50
        y = y - 50
        h = h + 50
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
#         roi_gray = gray[y:y+h, x:x+w]
#         roi_color = img[y:y+h, x:x+w]
        eyes = eye_classifier.detectMultiScale(gray)
        
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(img,(ex,ey),(ex+ew,ey+eh),(0,0,255),2) 

# 좌우반전을 줬는데, cam 원본이 좌우반전이 아니라서 오히려 화면이 어지러움
#     img = cv2.flip(img,1)
# 안경을 쓰면 제대로 인식을 못한다. 보완할 수 있을까?
    return img

cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    cv2.imshow('Our Face Extractor', face_detector(frame))
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      


detectMultiScale의 파라미터 (직전 예시의 경우 1.3 과 5) 가 갖는 의미

### Tuning Cascade Classifiers

*ourClassifier*.**detectMultiScale**(input image, **Scale Factor** , **Min Neighbors**)

- **Scale Factor**
Specifies how much we reduce the image size each time we scale. E.g. in face detection we typically use 1.3. This means we reduce the image by 30% each time it’s scaled. Smaller values, like 1.05 will take longer to compute, but will increase the rate of detection.

단순히 생각하면, 숫자가 작을수록 more precise한 경향을 보인다. face bounding box도 얼굴에 훨씬 근접해지고, eye box도 더 많이 나타난다. 실제 detection한 bounding box의 30% 정도 크기를 더 키운다고 보면 될 듯. 값을 1로 주면 효과가 없다.



- **Min Neighbors**
Specifies the number of neighbors each potential window should have in order to consider it a positive detection. 

다시말해, potential window가 window로 인정받기 위해서 각 window간 distance의 최소치. same region에 bounding box가 최소 몇 개는 그려져야 해당 region을 인식할 것인지. 따라서 low value = detect multiple pieces of a single face. high value = reduce the sensitivity of detection units / algorithm.

Typically set between 3-6. 
It acts as sensitivity setting, low values will sometimes detect multiples faces over a single face. High values will ensure less false positives, but you may miss some faces.  


## Car & Pedistrian Detection

### Pedistrian Detection



In [16]:
import cv2
import numpy as np

# Create our body classifier
body_classifier = cv2.CascadeClassifier('./MasteringComputerVision-V1.03/Master OpenCV/Haarcascades/haarcascade_fullbody.xml')

# Initiate video capture for video file
cap = cv2.VideoCapture('./MasteringComputerVision-V1.03/Master OpenCV/images/walking.avi')

# Loop once video is successfully loaded
while cap.isOpened():
    
    # Read first frame
    ret, frame = cap.read()
    # resize를 하는 이유는 speed up classification. resolution을 절반으로 줄이고 interpolation 방법으로 Inter linear 사용.
    frame = cv2.resize(frame, None,fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Pass frame to our body classifier
    bodies = body_classifier.detectMultiScale(gray, 1.2, 3)
    
    # Extract bounding boxes for any bodies identified
    for (x,y,w,h) in bodies:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
        cv2.imshow('Pedestrians', frame)

    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()

In [21]:
import cv2
import time
import numpy as np

# Create our body classifier
car_classifier = cv2.CascadeClassifier('./MasteringComputerVision-V1.03/Master OpenCV/Haarcascades/haarcascade_car.xml')

# Initiate video capture for video file
cap = cv2.VideoCapture('./MasteringComputerVision-V1.03/Master OpenCV/images/cars.avi')


# Loop once video is successfully loaded
while cap.isOpened():
    
    time.sleep(.05)
    # Read first frame
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
   
    # Pass frame to our car classifier
    cars = car_classifier.detectMultiScale(gray, 1.3, 2)
    
    # Extract bounding boxes for any bodies identified
    for (x,y,w,h) in cars:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
        cv2.imshow('Cars', frame)

    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()